In [14]:
# 导入组件 Importing Components
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=640, height=640)  #设置摄像头显示组件  Set up the camera display component

# 导入cv相关库 Import cv related libraries
import cv2
import numpy as np
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
# 导入依赖包 Import dependency packages
import hyperlpr3 as lpr3
import time

In [15]:
# 将BGR图像转换为JPEG格式的字节流 Convert a BGR image to a JPEG byte stream
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 在图像上绘制车牌框及文字 Draw the license plate frame and text on the image
def draw_plate_on_image(img, box, text, font):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (225, 32, 39), 2, cv2.LINE_AA)
    cv2.rectangle(img, (x1, y1 - 20), (x2, y1), (225, 32, 39), -1)
    data = Image.fromarray(img)
    draw = ImageDraw.Draw(data)
    draw.text((x1 + 1, y1 - 18), text, (255, 255, 255), font=font)
    res = np.asarray(data)
    return res


In [16]:
import cv2
import os,socket,sys,time
import spidev as SPI
import xgoscreen.LCD_2inch as LCD_2inch
from PIL import Image,ImageDraw,ImageFont
import numpy as np
import mediapipe as mp
from numpy import linalg
from xgolib import XGO

In [17]:
g_car = XGO(port='/dev/ttyAMA0',version="xgolite")
fm=g_car.read_firmware()
if fm[0]=='M':
    print('XGO-MINI')
    g_car = XGO(port='/dev/ttyAMA0',version="xgomini")
    dog_type='M'
elif fm[0]=='L':
    print('XGO-LITE')
    dog_type='L'
elif fm[0]=='R':
    print('XGO-RIDER')
    g_car = XGO(port='/dev/ttyAMA0',version="xgorider")
    dog_type='R'

XGO-LITE


In [18]:
#清屏
mydisplay = LCD_2inch.LCD_2inch()
mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)

In [19]:
try:
    code=0
    confidence=0
    type_idx=0
    box=0
    image=0
    display(image_widget)
    # 中文字体加载 Chinese font loading
    font_ch = ImageFont.truetype("platech.ttf", 20, 0)
    # 实例化识别对象 Instantiate the recognition object
    catcher = lpr3.LicensePlateCatcher(detect_level=lpr3.DETECT_LEVEL_LOW)#DETECT_LEVEL_HIGH640*640
    
    camera = cv2.VideoCapture(0)     # 定义摄像头对象，参数0表示第一个摄像头，默认640x480 Define the camera object. Parameter 0 indicates the first camera. The default resolution is 640x480.
    camera.set(3, 320)
    camera.set(4, 240)
    pTime, cTime = 0, 0
    while True:
        ret, frame = camera.read()
        # 执行识别算法
        results = catcher(frame)
        # 计算帧率
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        text = "FPS : " + str(int(fps))
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        # 初始化图像变量 Initialize image variables
        image = frame.copy()  # 使用原始帧作为默认图像 Use original frame as default image
        for code, confidence, type_idx, box in results:
                text = f"{code} - {confidence:.2f}"
                image = draw_plate_on_image(frame, box, text, font=font_ch)
       
        if results and len(results) > 0:
            code, confidence, _, _ = results[0]
            carcher_str = f'carcher : {code}'
            confidence_str = f'confidence: {confidence:.2f}'
        
        image_widget.value = bgr8_to_jpeg(image)

        #显示在小车的lcd屏幕上
        b,g,r = cv2.split(frame)
        img = cv2.merge((r,g,b))
        imgok = Image.fromarray(img)
        mydisplay.ShowImage(imgok)
        

        # cv2.imshow('frame', frame)
        cher_list = results[0] if results and results[0] is not None else None
        if cher_list is not None:
            print(cher_list)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

except KeyboardInterrupt:
    # picam2.stop()
    # picam2.close()
    camera.release()

Image(value=b'', format='jpeg', height='640', width='640')

[ WARN:0@81.202] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@81.207] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


AttributeError: 'NoneType' object has no attribute 'shape'

In [21]:
#最后需要释放掉摄像头的占用 Finally, you need to release the camera's occupancy
camera.release()

mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)

del g_car

NameError: name 'g_car' is not defined